This file will be used to contain data processing components

In [ ]:
%pip install boto3 pyspark delta-spark python-dotenv

In [1]:
import os
from pyspark.sql import SparkSession

In [25]:
os.environ['OBJ_STORAGE_ACCESS_KEY'] = 'WlsvIS3bB1xYKjbIZMsy'
os.environ['OBJ_STORAGE_SECRET_KEY'] = 'BibSNsZ0y97zUXZvyiaidU1UAWf5ORBTyOC6L8B5'
os.environ['OBJ_STORAGE_ENDPOINT'] = 'http://localhost:9000'

In [3]:
minio_folder_path = "data/data-result"

path_1 = "s3a://data/data-raw/data.json"
path_2 = "s3a://data/data-raw/data2.json"
path_3 = "s3a://data/data-raw/data3.json"

In [4]:
import os
from pyspark.sql import SparkSession

obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT')


In [26]:
# You need to more configuration if you want to use minio as object storage 
# (hint: maybe you can using .config() method to set the configuration if you want using spark to read/write data from/to minio)
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.hadoop.fs.s3a.access.key", obj_storage_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", obj_storage_secret_key) \
    .config("spark.hadoop.fs.s3a.endpoint", obj_storage_endpoint) \
    .config('spark.hadoop.fs.s3a.path.style.access', "true") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.4.0") \
    .getOrCreate()

In [34]:
file_1 = spark.read.json(path_1, multiLine=True)
file_2 = spark.read.json(path_2, multiLine=True)
file_3 = spark.read.json(path_3, multiLine=True)

In [35]:
file_merged = file_1.union(file_2).union(file_3)
file_merged.show()

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|{[{1001, Regular}...|                NULL|0004|              Jelly|0.65|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0005|     Custard-Filled|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|[{6001, None}, {6...|0006|     Cinnamon Twist|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0007|    Vanilla Frosted|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0008| Strawberry Frosted|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0009|     Chocolate Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0010|     Blueberry Cake|0.55|[{5001, None}, {5.

In [36]:
file_merged.write.format("json").mode("overwrite").save("s3a://data/data-result/result")

In [37]:
df = spark.read.format("json").load("s3a://data/data-result/result")
df.show()

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|{[{1001, Regular}...|                NULL|0004|              Jelly|0.65|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0005|     Custard-Filled|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|[{6001, None}, {6...|0006|     Cinnamon Twist|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0007|    Vanilla Frosted|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0008| Strawberry Frosted|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0009|     Chocolate Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0010|     Blueberry Cake|0.55|[{5001, None}, {5.

### Remove Duplicate

In [39]:
file_merged = file_merged.dropDuplicates()

In [40]:
file_merged.write.format("parquet").mode("overwrite").save("s3a://data/data-result/result")

In [41]:
df = spark.read.format("parquet").load("s3a://data/data-result/result")
df.show()

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|{[{1001, Regular}...|                NULL|0004|              Jelly|0.65|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0011|  Devil's Food Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0010|     Blueberry Cake|0.55|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0007|    Vanilla Frosted|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|[{6001, None}, {6...|0006|     Cinnamon Twist|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0009|     Chocolate Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0005|     Custard-Filled|0.75|[{5001, None}, {5.